In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp "/content/drive/MyDrive/ColumbiaMSCS/COMS4995_DL_for_CV/Project/Data/project_data.zip" "/content/project_data.zip"

In [ ]:
!mkdir -p "/content/data"

In [ ]:
!unzip -q "/content/project_data.zip" -d "/content/data"

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset, random_split
from torchvision import models
from torchvision.models import googlenet, GoogLeNet_Weights
import numpy as np
import os
import shutil
import glob
import matplotlib.pyplot as plt
import random

In [ ]:
# Count of each class in the data
project_data = {}

project_data['spring'] = glob.glob('/content/data/project_data/spring/spring*')
project_data['summer'] = glob.glob('/content/data/project_data/summer/summer*')
project_data['fall'] = glob.glob('/content/data/project_data/fall/fall*')
project_data['winter'] = glob.glob('/content/data/project_data/winter/winter*')

print(f"count of spring images :  {len(project_data['spring'])}")
print(f"count of summer images :  {len(project_data['summer'])}")
print(f"count of fall images :  {len(project_data['fall'])}")
print(f"count of winter images :  {len(project_data['winter'])}")

count of spring images :  6000
count of summer images :  6000
count of fall images :  6000
count of winter images :  6000


In [ ]:
SOURCE = '/content/data/project_data'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
transform = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
dataset = datasets.ImageFolder(root=SOURCE, transform=transform)

In [ ]:
# Use 4,800 images of each category for traning, 600 of each category for validation, 600 images of each category for testing
fall_samples = []
spring_samples = []
summer_samples = []
winter_samples = []

# Separate dataset into four label groups
for i in range(len(dataset)):
    _, label = dataset[i]
    if label == 0:
        fall_samples.append(i)
    elif label == 1:
        spring_samples.append(i)
    elif label == 2:
        summer_samples.append(i)
    else:
        winter_samples.append(i)

# Shuffle the samples
random.shuffle(fall_samples)
random.shuffle(spring_samples)
random.shuffle(summer_samples)
random.shuffle(winter_samples)

# Split the samples
train_indices = fall_samples[:4800] + spring_samples[:4800] + summer_samples[:4800] + winter_samples[:4800]
val_indices = fall_samples[4800:5400] + spring_samples[4800:5400] + summer_samples[4800:5400] + winter_samples[4800:5400]
test_indices = fall_samples[5400:6000] + spring_samples[5400:6000] + summer_samples[5400:6000] + winter_samples[5400:6000]

# Shuffle the indices
random.shuffle(train_indices)
random.shuffle(val_indices)
random.shuffle(test_indices)

# Create subset datasets
train_dataset = Subset(dataset, train_indices)
val_dataset = Subset(dataset, val_indices)
test_dataset = Subset(dataset, test_indices)

In [ ]:
# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:
# Load pre-trained GoogLeNet
model_googlenet = googlenet(weights=GoogLeNet_Weights.DEFAULT)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth
100%|██████████| 49.7M/49.7M [00:00<00:00, 100MB/s]


In [ ]:
# Change the final fully connected layer's output feature size to 4 so that
# it's suitable for softmax activation for classification on four seasons
model_googlenet.fc.out_features = 4

In [ ]:
# Freeze all layers except the last modified layer
for name, param in model_googlenet.named_parameters():
    if "fc" not in name:
        param.requires_grad = False
    else:
        param.requires_grad = True

In [ ]:
# Use GPU
model_googlenet = model_googlenet.to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_googlenet.fc.parameters(), lr=0.001)
softmax = nn.Softmax(dim=1)

In [ ]:
# Define training function
def train_model(model, train_loader, val_loader, loss_fn, optimizer, epochs, threshold):
    best_val_loss = float('inf')
    degrade_times = 0
    for epoch in range(epochs):
        train_loss = 0.0
        train_corrects = 0
        train_count = 0
        model.train()
        for x_batch, y_batch in train_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            optimizer.zero_grad()
            outputs = model(x_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * len(y_batch)
            pred = softmax(outputs)
            train_corrects += (torch.argmax(pred, dim=1) == y_batch).float().sum()
            train_count += y_batch.size(0)
        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_corrects / train_count

        val_loss = 0.0
        val_corrects = 0
        val_count = 0
        model.eval()
        with torch.no_grad():
            for x_batch, y_batch in val_loader:
                x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
                outputs = model(x_batch)
                loss = loss_fn(outputs, y_batch)
                val_loss += loss.item() * len(y_batch)
                pred = softmax(outputs)
                val_corrects += (torch.argmax(pred, dim=1) == y_batch).float().sum()
                val_count += y_batch.size(0)
        val_loss = val_loss / len(val_loader.dataset)
        val_acc = val_corrects / val_count
        print(f'Epoch {epoch} Train Loss {train_loss:.4f} Train Accuracy {train_acc:.4f} Validation Loss {val_loss:.4f} Validation Accuracy {val_acc:.4f}')

        # Check for early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            degrade_times = 0
            # Save the model if it has the best validation loss so far
            torch.save(model.state_dict(), './best_model_googlenet.pth')
        else:
            degrade_times += 1
            # If the number of epochs where validation loss continuously increases
            # is larger than threshold, stop training the network to avoid overfitting
            if degrade_times > threshold:
                print(f'Early stopping at epoch {epoch}')
                break

In [ ]:
# Define evaluation function
def evaluate_model(model, test_loader):
    correct = 0
    count = 0
    model.eval()
    with torch.no_grad():
        for x_batch, y_batch in test_loader:
            x_batch, y_batch = x_batch.to(DEVICE), y_batch.to(DEVICE)
            outputs = model(x_batch)
            pred = softmax(outputs)
            correct += (torch.argmax(pred, dim=1) == y_batch).float().sum()
            count += y_batch.size(0)
        test_acc = correct / count
    print(f'Accuracy on test set: {test_acc:.4f}')

In [ ]:
# Fine-tune the network on seasons image data
train_model(model_googlenet, train_loader, val_loader, loss_fn, optimizer, epochs=10, threshold=3)

Epoch 0 Train Loss 1.1269 Train Accuracy 0.5717 Validation Loss 0.8196 Validation Accuracy 0.6713
Epoch 1 Train Loss 0.8613 Train Accuracy 0.6627 Validation Loss 0.7712 Validation Accuracy 0.6896
Epoch 2 Train Loss 0.8277 Train Accuracy 0.6726 Validation Loss 0.7527 Validation Accuracy 0.7013
Epoch 3 Train Loss 0.8112 Train Accuracy 0.6806 Validation Loss 0.7494 Validation Accuracy 0.6917
Epoch 4 Train Loss 0.8157 Train Accuracy 0.6778 Validation Loss 0.7485 Validation Accuracy 0.7013
Epoch 5 Train Loss 0.8137 Train Accuracy 0.6817 Validation Loss 0.7451 Validation Accuracy 0.7033
Epoch 6 Train Loss 0.8099 Train Accuracy 0.6833 Validation Loss 0.7488 Validation Accuracy 0.7021
Epoch 7 Train Loss 0.8045 Train Accuracy 0.6802 Validation Loss 0.7386 Validation Accuracy 0.7004
Epoch 8 Train Loss 0.8062 Train Accuracy 0.6846 Validation Loss 0.7408 Validation Accuracy 0.6958
Epoch 9 Train Loss 0.8105 Train Accuracy 0.6826 Validation Loss 0.7534 Validation Accuracy 0.6963


In [ ]:
# Load the best model
model_googlenet.load_state_dict(torch.load('./best_model_googlenet.pth'))
# Evaluate on the test set
evaluate_model(model_googlenet, test_loader)

Accuracy on test set: 0.6979


In [ ]:
# Unfreeze all layers
for param in model_googlenet.parameters():
    param.requires_grad = True

In [ ]:
# Set the optimizer with a smaller learning rate
optimizer = optim.Adam(model_googlenet.parameters(), lr=0.0005)
# Continue fine-tuning the network
train_model(model_googlenet, train_loader, val_loader, loss_fn, optimizer, epochs=10, threshold=3)

Epoch 0 Train Loss 0.7905 Train Accuracy 0.6906 Validation Loss 0.6972 Validation Accuracy 0.7254
Epoch 1 Train Loss 0.6715 Train Accuracy 0.7414 Validation Loss 0.7344 Validation Accuracy 0.7121
Epoch 2 Train Loss 0.6042 Train Accuracy 0.7670 Validation Loss 0.7314 Validation Accuracy 0.7096
Epoch 3 Train Loss 0.5623 Train Accuracy 0.7839 Validation Loss 0.7185 Validation Accuracy 0.7221
Epoch 4 Train Loss 0.5030 Train Accuracy 0.8081 Validation Loss 0.7270 Validation Accuracy 0.7304
Early stopping at epoch 4


In [ ]:
# Load the best model
model_googlenet.load_state_dict(torch.load('./best_model_googlenet.pth'))
# Evaluate on the test set
evaluate_model(model_googlenet, test_loader)

Accuracy on test set: 0.7183
